# PART ONE

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import requests

from bs4 import BeautifulSoup

Load URL and Load DataFrame

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

toronto_df = pd.read_html(url, header=0)[0]

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
toronto_df.drop(toronto_df[toronto_df['Borough'] == "Not assigned"].index, axis=0, inplace=True)

toronto_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

In [4]:
toronto_df = toronto_df.groupby("Postcode").agg(lambda x:','.join(set(x)))

toronto_df.head(20)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview"
M1L,Scarborough,"Oakridge,Golden Mile,Clairlea"
M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [5]:
toronto_df.loc[toronto_df['Neighbourhood']=="Not assigned",'Neighbourhood']= \
toronto_df.loc[toronto_df['Neighbourhood']=="Not assigned",'Borough']
toronto_df.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


_Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making._

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [6]:
print ("no. rows= ", toronto_df.shape[0])

no. rows=  103


# PART TWO - Toronto Coordinates 

Load additional libraries

In [7]:
from geopy.geocoders import Nominatim

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')
print('Data downloaded!')


Solving environment: done

# All requested packages already installed.

Libraries imported.
Data downloaded!


Load Toronto Coordinates

In [8]:
toronto_coordinates = pd.read_csv("https://cocl.us/Geospatial_data")

toronto_df ['Latitude'] = toronto_coordinates ['Latitude'].values

toronto_df['Longitude'] = toronto_coordinates ['Longitude'].values

toronto_df.head()

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="foursquare_agent") 

location = geolocator.geocode(address) 

latitude = location.latitude 

longitude = location.longitude 

print('Data downloaded!')
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Data downloaded!
The geographical coordinate of Toronto are 43.653963, -79.387207.


In [9]:
toronto_df

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview",43.727929,-79.262029
M1L,Scarborough,"Oakridge,Golden Mile,Clairlea",43.711112,-79.284577
M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West",43.716316,-79.239476


# PART THREE - Exploring Toronto

Create Map of Toronto Using Latitude and Longtitude Values

In [10]:
!pip install folium

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
    
map_toronto

Information about Foursquare

In [41]:
CLIENT_ID = '0PV4LA2UFQZUDGSHWMXMJ2ZKEFAV0PYZL01P15MBSJWOTVA5'
CLIENT_SECRET = '5SVO3OBC4U0PNJTO2QEUBSZPOD5XJGMXGKR21J3LEPAYDISU'
VERSION = '20180605' # Foursquare API version
LIMIT = 50 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0PV4LA2UFQZUDGSHWMXMJ2ZKEFAV0PYZL01P15MBSJWOTVA5
CLIENT_SECRET:5SVO3OBC4U0PNJTO2QEUBSZPOD5XJGMXGKR21J3LEPAYDISU


In [42]:
search_query = address
radius = 500

torontosearch_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
torontosearch_url

'https://api.foursquare.com/v2/venues/search?client_id=0PV4LA2UFQZUDGSHWMXMJ2ZKEFAV0PYZL01P15MBSJWOTVA5&client_secret=5SVO3OBC4U0PNJTO2QEUBSZPOD5XJGMXGKR21J3LEPAYDISU&ll=43.653963,-79.387207&v=20180605&query=Toronto, CA&radius=500&limit=50'

In [43]:
results = requests.get(torontosearch_url).json()
results

{'meta': {'code': 200, 'requestId': '5d7ba555bcbf7a002cd52edd'},
 'response': {'venues': [{'id': '598c9619c0cacb28d835bb4f',
    'name': 'Toronto Canada On Fire',
    'location': {'lat': 43.652672,
     'lng': -79.38111,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.652672,
       'lng': -79.38111}],
     'distance': 511,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Toronto ON', 'Canada']},
    'categories': [{'id': '52e81612bcbc57f1066b79ee',
      'name': 'Street Art',
      'pluralName': 'Street Art Installations',
      'shortName': 'Street Art',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1568384341',
    'hasPerk': False},
   {'id': '5b2f4aa2f5e9d7002c10bd6e',
    'name': 'Toronto Car Service',
    'location': {'address': '70 Temperance St',
     'lat': 43.650576,
     'lng':

In [44]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52e81612bcbc57f1066b79ee', 'name': 'S...",False,598c9619c0cacb28d835bb4f,NaN,CA,Toronto,Canada,NaN,511,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.652672, 'lng':...",43.652672,-79.381110,NaN,NaN,ON,Toronto Canada On Fire,v-1568384341,NaN
1,"[{'id': '52f2ab2ebcbc57f1066b8b44', 'name': 'A...",False,5b2f4aa2f5e9d7002c10bd6e,70 Temperance St,CA,Toronto,Canada,NaN,552,"[70 Temperance St, Toronto ON M5H 0B1, Canada]","[{'label': 'display', 'lat': 43.650576, 'lng':...",43.650576,-79.382201,NaN,M5H 0B1,ON,Toronto Car Service,v-1568384341,NaN
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad77a12f964a520260b21e3,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,535,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654265,-79.380567,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,CF Toronto Eaton Centre,v-1568384341,NaN
3,"[{'id': '50aa9e094b90af0d42d5de0d', 'name': 'C...",False,4c50d7d7250dd13a12fa377c,NaN,CA,NaN,Canada,NaN,508,"[Ontario, Canada]",NaN,43.650072,-79.383888,NaN,NaN,Ontario,City of Toronto,v-1568384341,NaN
4,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,4ad4c05ef964a5208ff620e3,100 Queen St. W.,CA,Toronto,Canada,at Bay St.,276,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...","[{'label': 'display', 'lat': 43.65313989695342...",43.653140,-79.383967,NaN,M5H 2N2,ON,Toronto City Hall,v-1568384341,NaN


In [45]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Toronto Canada On Fire,Street Art,NaN,CA,Toronto,Canada,NaN,511,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.652672, 'lng':...",43.652672,-79.381110,NaN,NaN,ON,598c9619c0cacb28d835bb4f
1,Toronto Car Service,Auto Garage,70 Temperance St,CA,Toronto,Canada,NaN,552,"[70 Temperance St, Toronto ON M5H 0B1, Canada]","[{'label': 'display', 'lat': 43.650576, 'lng':...",43.650576,-79.382201,NaN,M5H 0B1,ON,5b2f4aa2f5e9d7002c10bd6e
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,535,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654265,-79.380567,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,4ad77a12f964a520260b21e3
3,City of Toronto,City,NaN,CA,NaN,Canada,NaN,508,"[Ontario, Canada]",NaN,43.650072,-79.383888,NaN,NaN,Ontario,4c50d7d7250dd13a12fa377c
4,Toronto City Hall,City Hall,100 Queen St. W.,CA,Toronto,Canada,at Bay St.,276,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...","[{'label': 'display', 'lat': 43.65313989695342...",43.653140,-79.383967,NaN,M5H 2N2,ON,4ad4c05ef964a5208ff620e3
5,Toronto Coach Terminal,Bus Station,610 Bay St,CA,Toronto,Canada,at Edward St,339,"[610 Bay St (at Edward St), Toronto ON M5G 1M5...","[{'label': 'display', 'lat': 43.65625846099937...",43.656258,-79.384425,NaN,M5G 1M5,ON,4b430db1f964a520e5db25e3
6,Downtown Toronto,Neighborhood,NaN,CA,Toronto,Canada,NaN,174,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65323167517444...",43.653232,-79.385296,NaN,NaN,ON,5227bb01498e17bf485e6202
7,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,CA,Toronto,Canada,at York St.,390,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6511441, 'lng'...",43.651144,-79.384329,NaN,M5H 2M9,ON,4ab2d511f964a5209b6c20e3
8,GoodLife Fitness Toronto McCaul and Queen,Gym,21 McCaul St,CA,Toronto,Canada,at Queen St W,371,"[21 McCaul St (at Queen St W), Toronto ON M5T ...","[{'label': 'display', 'lat': 43.651148, 'lng':...",43.651148,-79.389683,NaN,M5T 1V7,ON,4af8a8e5f964a520030f22e3
9,Toronto General Hospital,Hospital,190 Elizabeth St,CA,Toronto,Canada,at Gerrard St W,541,"[190 Elizabeth St (at Gerrard St W), Toronto O...","[{'label': 'display', 'lat': 43.65876218215914...",43.658762,-79.388292,NaN,M5G 2C4,ON,4ad4c064f964a52070f820e3


In [46]:
dataframe_filtered.name

0                                Toronto Canada On Fire
1                                   Toronto Car Service
2                               CF Toronto Eaton Centre
3                                       City of Toronto
4                                     Toronto City Hall
5                                Toronto Coach Terminal
6                                      Downtown Toronto
7                         Sheraton Centre Toronto Hotel
8             GoodLife Fitness Toronto McCaul and Queen
9                              Toronto General Hospital
10                                       Hilton Toronto
11         Marriott Downtown at CF Toronto Eaton Centre
12                                   Shangri-La Toronto
13                     Toronto Rehabilitation Institute
14                               Toronto Public Library
15          ILSC-Toronto Chinese Catholic Centre Campus
16                          Condo Move | Toronto Condos
17                      Toronto Civil Data Entry

In [47]:
# generate map centred around the Downtown Toronto

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent the Downtown Toronto
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


# display map
venues_map